In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import preprocess_input
from keras.applications.nasnet import NASNetLarge
from keras.applications.xception import Xception
from keras.applications.resnet50 import ResNet50
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Sequential
from mpl_toolkits.axes_grid1 import ImageGrid
from keras.applications.vgg16 import VGG16
import matplotlib.pyplot as plt
from keras.models import Model
from keras import applications
import tensorflow as tf
from glob import glob
import numpy as np
import matplotlib

In [ ]:
#Data Path
train_path = '/content/drive/MyDrive/DL_Assignment 2/train'
test_path = '/content/drive/MyDrive/DL_Assignment 2/test'
val_path = '/content/drive/MyDrive/DL_Assignment 2/val'

image_files = glob(train_path + '/*/*.jp*g')
val_image_files = glob(val_path + '/*/*.jp*g')

folders = glob(train_path + '/*')
num_classes = len(folders)

image_size = [224, 224]
nas_image_size = [331, 331]
batch_size = 32

In [ ]:
def Get_model_inputs(Model):
  if Model == 'VGG16':
    from keras.applications.vgg16 import preprocess_input
    train_datagen = ImageDataGenerator(rescale = 1./255, 
                                      shear_range = 0.2,
                                      zoom_range = 0.2,
                                      horizontal_flip = True,
                                      preprocessing_function=preprocess_input)
    test_datagen = ImageDataGenerator(rescale = 1./255,
                                      preprocessing_function=preprocess_input)
    val_datagen = ImageDataGenerator(rescale = 1./255,
                                     preprocessing_function=preprocess_input)

  if Model == 'ResNet50':
    from keras.applications.resnet50 import preprocess_input
    train_datagen = ImageDataGenerator(rescale = 1./255, 
                                      shear_range = 0.2,
                                      zoom_range = 0.2,
                                      horizontal_flip = True,
                                      preprocessing_function=preprocess_input)
    test_datagen = ImageDataGenerator(rescale = 1./255,
                                      preprocessing_function=preprocess_input)
    val_datagen = ImageDataGenerator(rescale = 1./255,
                                     preprocessing_function=preprocess_input)
    
  if Model == 'InceptionV3':
    from keras.applications.inception_v3 import preprocess_input
    train_datagen = ImageDataGenerator(rescale = 1./255, 
                                      shear_range = 0.2,
                                      zoom_range = 0.2,
                                      horizontal_flip = True,
                                      preprocessing_function=preprocess_input)
    test_datagen = ImageDataGenerator(rescale = 1./255,
                                      preprocessing_function=preprocess_input)
    val_datagen = ImageDataGenerator(rescale = 1./255,
                                      preprocessing_function=preprocess_input)
    
  if Model == 'Inception_Resnet_V2':
    from keras.applications.inception_resnet_v2 import preprocess_input
    train_datagen = ImageDataGenerator(rescale = 1./255, 
                                      shear_range = 0.2,
                                      zoom_range = 0.2,
                                      horizontal_flip = True,
                                      preprocessing_function=preprocess_input)
    test_datagen = ImageDataGenerator(rescale = 1./255,
                                      preprocessing_function=preprocess_input)
    val_datagen = ImageDataGenerator(rescale = 1./255,
                                      preprocessing_function=preprocess_input)
    
  if Model == 'Xception':
    from keras.applications.xception import preprocess_input
    train_datagen = ImageDataGenerator(rescale = 1./255, 
                                      shear_range = 0.2,
                                      zoom_range = 0.2,
                                      horizontal_flip = True,
                                      preprocessing_function=preprocess_input)
    test_datagen = ImageDataGenerator(rescale = 1./255,
                                      preprocessing_function=preprocess_input)
    val_datagen = ImageDataGenerator(rescale = 1./255,
                                      preprocessing_function=preprocess_input)
    
  if Model == 'NASNetLarge':
    from keras.applications.nasnet import preprocess_input
    train_datagen = ImageDataGenerator(rescale = 1./255, 
                                      shear_range = 0.2,
                                      zoom_range = 0.2,
                                      horizontal_flip = True,
                                      preprocessing_function=preprocess_input)
    test_datagen = ImageDataGenerator(rescale = 1./255,
                                      preprocessing_function=preprocess_input)
    val_datagen = ImageDataGenerator(rescale = 1./255,
                                      preprocessing_function=preprocess_input)

  if Model == 'NASNetLarge':
    train = train_datagen.flow_from_directory(
            train_path,
            target_size = nas_image_size,
            shuffle = True,
            batch_size = 32,
            class_mode = 'categorical')

    test = test_datagen.flow_from_directory(
              test_path,
              target_size = nas_image_size,
              shuffle = True,
              batch_size = 32,
              class_mode = 'categorical')
              

    val = val_datagen.flow_from_directory(
            val_path,
            target_size = nas_image_size,
            shuffle = True,
            batch_size = 32,
            class_mode = 'categorical')
  else:
    train = train_datagen.flow_from_directory(
            train_path,
            target_size = image_size,
            shuffle = True,
            batch_size = 32,
            class_mode = 'categorical')

    test = test_datagen.flow_from_directory(
              test_path,
              target_size = image_size,
              shuffle = True,
              batch_size = 32,
              class_mode = 'categorical')
              

    val = val_datagen.flow_from_directory(
            val_path,
            target_size = image_size,
            shuffle = True,
            batch_size = 32,
            class_mode = 'categorical')
  
  train.class_indices
  return train, val, test

In [ ]:
def Xception_call():
  xception = Xception(input_shape = image_size + [3], weights = 'imagenet', include_top = False)

  for layer in xception.layers:
      layer.trainable = False

  class myCallback(tf.keras.callbacks.Callback):
     def on_epoch_end(self, epoch, logs={}):
       if(logs.get('accuracy') > 0.959):
          print("\nReached 95.9% accuracy so cancelling training!")
          self.model.stop_training = True

  out = Flatten()(xception.output)
  out = Dense(1024, activation='relu')(out)
  out = Dropout(0.2)(out)  
  out = Dense(num_classes, activation = 'softmax')(out)
  model = Model(inputs = xception.input, outputs = out)
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.summary()
  train, val, test = Get_model_inputs('Xception')
  
  callbacks = myCallback()
  history = model.fit(train,
                   batch_size = batch_size,  
                   epochs = 10, 
                   validation_data = val,
                   callbacks=[callbacks]
                   )
  return history.history['accuracy'], history.history['val_accuracy']

In [ ]:
Train_acc , val_acc = Xception_call()
print ('Training Accuracy = ' + str(Train_acc))
print ('Validation Accuracy = ' + str(val_acc))

In [ ]:
def NASNet_call():
  nasnetl = NASNetLarge(input_shape = (331, 331, 3), weights = 'imagenet', include_top = False)

  for layer in nasnetl.layers:
      layer.trainable = False

  class myCallback(tf.keras.callbacks.Callback):
     def on_epoch_end(self, epoch, logs={}):
       if(logs.get('accuracy') > 0.959):
          print("\nReached 95.9% accuracy so cancelling training!")
          self.model.stop_training = True

  out = Flatten()(nasnetl.output)
  #out = Dense(1024, activation='relu')(out)
  #out = Dropout(0.2)(out)  
  out = Dense(num_classes, activation = 'softmax')(out)
  model = Model(inputs = nasnetl.input, outputs = out)
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.summary()
  train, val, test = Get_model_inputs('NASNetLarge')

  callbacks = myCallback()
  history = model.fit(train,
                   batch_size = batch_size,  
                   epochs = 10, 
                   validation_data = val,
                   callbacks=[callbacks]
                   )
  return history.history['accuracy'], history.history['val_accuracy']

In [ ]:
Train_acc , val_acc = NASNet_call()
print ('Training Accuracy = ' + str(Train_acc))
print ('Validation Accuracy = ' + str(val_acc))

Model: "model_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, 331, 331, 3) 0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 165, 165, 96) 2592        input_14[0][0]                   
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 165, 165, 96) 384         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation_3380 (Activation)    (None, 165, 165, 96) 0           stem_bn1[0][0]                   
___________________________________________________________________________________________